# import

In [23]:
## import functions from .py utils file: 
from Utilities.preprocessing import *
import os
from IPython.display import display, HTML
from tqdm import tqdm 
from datetime import datetime


# paths

In [41]:
InputPath = r"Z:\projects\sequence_squad\revision_data\lars_recordings\bpod_raw_data\\"
# InputPath = r"D:\EPHYS_BPOD_data_example\\"

out_path_base = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"

Replace = True



# set paths

In [42]:

# find main paths for the animals
main_paths = []
Animal_ID = []
Animal_save_paths = []
Animal_bpod_save_paths = []
Animal_ephys_dates = []

# for each animal in the bpod dump path 
for mouse in os.listdir(InputPath):
    if not mouse in  ['FakeSubject','SEQ037','SEQ038']:
        save_paths = []
        # find dates that match an ephys recording in teh organised folder
        for animal_file in os.listdir(out_path_base):
            # for seq00 animals
            if mouse.lower() in animal_file:
                recordings = os.listdir(os.path.join(out_path_base,animal_file))
                # Sort by date
                sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                save_paths = list(os.path.join(out_path_base,animal_file,recording) for recording in sorted_recordings)
                break
                ## extract date from recordings
           
            elif 'ap5' in mouse.lower():
                if 'lr' in mouse.lower():
                    recordings = os.listdir(os.path.join(out_path_base,'ap5lr_implant1'))
                    # Sort by date
                    sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                    ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                    save_paths = list(os.path.join(out_path_base,'ap5lr_implant1',recording) for recording in sorted_recordings)
                    break
                elif 'L' in mouse:
                    recordings = os.listdir(os.path.join(out_path_base,'ap5L_implant1'))
                    # Sort by date
                    sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                    ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                    save_paths = list(os.path.join(out_path_base,'ap5L_implant1',recording) for recording in sorted_recordings)
                    break
                elif 'R' in mouse:
                    recordings = os.listdir(os.path.join(out_path_base,'ap5R_implant1'))
                    # Sort by date
                    sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                    ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                    save_paths = list(os.path.join(out_path_base,'ap5R_implant1',recording) for recording in sorted_recordings)
                    break
            elif mouse in animal_file:
                recordings = os.listdir(os.path.join(out_path_base,animal_file))
                # Sort by date
                sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                save_paths = list(os.path.join(out_path_base,animal_file,recording) for recording in sorted_recordings)
                break

        if len(save_paths) > 0: # ignore for files that are not in ephys organised folder
            # create full save paths 
            full_save_paths = []
            for save_path in save_paths:
                new_path = os.path.join(save_path,r'behav_sync/2_task/')
                if not os.path.isdir(new_path):
                    print('making path')
                    print(new_path)
                    os.makedirs(new_path)
                full_save_paths+=[new_path]
            Animal_bpod_save_paths += [full_save_paths]
            Animal_save_paths += [save_paths]
            Animal_ephys_dates += [ephys_dates]
            Animal_ID += [mouse]
            main_paths+=[os.path.join(InputPath,mouse,r'Sequence_Automated','Session Data//')]
        
      


In [40]:
Animal_ID

[]

In [43]:
main_paths

['Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq052\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq054\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq059\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq061\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq120\\Sequence_Automated\\Session Data//']

In [44]:
        
#### MAIN ####
for Animal_index, CurrentAnimal in enumerate(Animal_ID):
    # if CurrentAnimal == 'SP156':
        
    display(HTML(f"<b>Processing data for: {CurrentAnimal}</b>"))
    
    #Load Behavioural data:
    Behav_data, Sessions,Path,FileDates = Import_EPHYS_Bpod_DataFiles(main_paths[Animal_index],Animal_ephys_dates[Animal_index])
    
    Processed = ('')
    Skipped = ('')


    for Session in tqdm(range(0, Sessions)):

        print(Session)

        filedate = FileDates[Session] + '_' + str(Behav_data[Session]['__header__'])[-25:-22]

        if Session < 10:
            Save_path = (Animal_save_paths[Animal_index][Session] + r'\\'+ r'\behav_sync\2_task\Preprocessed' + r'\\0' + str(Session) + '_' + filedate)
        else:
            Save_path = (Animal_save_paths[Animal_index][Session] + r'\\' + r'\behav_sync\2_task\Preprocessed' + r'\\' + str(Session) + '_' + filedate)
            
            
        print(Save_path)

        # see if dir exists already then use bool logic to detemrine if it needs to be processed or not
        Process = check_and_create_directory(Save_path, Replace)

        if Process == True:
        
            #Convert to python friendly format:
            convert_nested_structs(Behav_data[Session])

        #    # Extract GUI info
            Trial_settings = todict(Behav_data[Session]['SessionData']['TrialSettings'][0])
            FinalRewardAmount = []
            for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                TLevel = item
                FinalRewardAmount = FinalRewardAmount + [Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][4]]

            # save out training levels on their own
            filename = 'Preprocessed_TrainingLevels' 
            with open(Save_path + r'\\'+ filename, 'wb') as fp:
                pickle.dump(Behav_data[Session]['SessionData']['SessionVariables']['TLevel'], fp)

            # save out led intensites and reward amounts on their own:
            LED_Intensities = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port1'],
                                            'Port2':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port2'],
                                            'Port3':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port3'],
                                            'Port4':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port4'],
                                            'Port5':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port5']})
            LED_Intensities.to_csv(Save_path + '/Preprocessed_LED_Intensities.csv')
            RewardAmounts = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port1'],
                                        'Port2':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port2'],
                                        'Port3':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port3'],
                                        'Port4':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port4'],
                                        'Port5':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port5']})
            RewardAmounts.to_csv(Save_path + '/Preprocessed_RewardAmounts.csv')


            #Extract PortIn times for each port and check for errors (inside this function):
            All_PortIn_Times,All_PortOut_Times,All_Port_references = extract_poke_times(Behav_data[Session])

            #remove nans (times when part [in or out poke] of the event was dropped for some reason by bpod)
            All_PortIn_Times_fixed,All_PortOut_Times_fixed ,All_Port_references_fixed = remove_dropped_in_events(All_PortIn_Times,All_PortOut_Times,All_Port_references)
            
            # Resort these in time:
            All_PortIn_Times_sorted,All_PortOut_Times_sorted,All_Port_references_sorted = time_sort(All_PortIn_Times,All_PortOut_Times,All_Port_references)

            #extract reward times:
            Reward_ts = extract_reward_times(Behav_data[Session])
            
            # find reward inds and align rewarded ts to poke events:
            Rewarded_event_inds = find_reward_inds(All_PortIn_Times_sorted,All_Port_references_sorted,Reward_ts)
            Reward_ts = np.asarray(Reward_ts)
            Reward_ts = Reward_ts[np.logical_not(np.isnan(Reward_ts))]
            Reward_ts = list(Reward_ts)
            Reward_ts_aligned = align_trigger_to_index(Reward_ts,Rewarded_event_inds,All_Port_references_sorted)

            #extract trial start time stamps
            Trial_start_ts = extract_trial_timestamps(Behav_data[Session])

            #extract trial end times:
            Trial_end_ts = extract_trial_end_times(Behav_data[Session])

            #determine trial IDs
            trial_id = determine_trial_id(All_PortIn_Times_sorted,Trial_end_ts)

            # align trial start and end times to poke events
            trialstart_index = find_trialstart_index(trial_id)
            trial_start_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_start_ts)
            trial_end_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_end_ts)

            #determine LED and reward states for each trial and align them to trials:
            IntermediateRewards = []
            LED_intensities = []
            for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                TLevel = item
                IntermediateRewards = IntermediateRewards + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][0:4])]
                LED_intensities = LED_intensities + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][6:10])]
            aligned_LED_intensities = align_trial_start_end_timestamps(trial_id,trialstart_index,LED_intensities)
            aligned_IntermediateRewards = align_trial_start_end_timestamps(trial_id,trialstart_index,IntermediateRewards)


            Trial_start_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)
            Trial_end_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)
            First_poke_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)

            ## align Training level for each trial:
            Training_Levels = align_opto_trials_to_dataframe(trial_id,Behav_data[Session]['SessionData']['SessionVariables']['TLevel'])

            # make portin dataframe:
            PortIn_df = pd.DataFrame(
                {'Trial_id' : trial_id,
                'Trial_Start' : trial_start_ts_aligned,
                'Port': All_Port_references_sorted,
                'PokeIn_Time': All_PortIn_Times_sorted,
                'PokeOut_Time': All_PortOut_Times_sorted,
                'Reward_Times': Reward_ts_aligned,
                'Trial_End' : trial_end_ts_aligned,
                'Port 2,3,4,5 LED intensities': aligned_LED_intensities,
                'Port 1,2,3,4 RewardAmount':aligned_IntermediateRewards,
                'BACK_Trial_Start_Camera_Time' : Trial_start_Camera_Ts_aligned,
                'BACK_Trial_End_Camera_Time' : Trial_end_Camera_Ts_aligned,
                'BACK_First_poke_Camera_Time' : First_poke_Camera_Ts_aligned,
                'TrainingLevel': Training_Levels})

            #Save Data
            PortIn_df.to_csv(Save_path +'/PreProcessed_RawPokeData.csv')


seq052_Sequence_Automated_20250617_111219.mat
seq052_Sequence_Automated_20250618_113251.mat
seq052_Sequence_Automated_20250710_095707.mat
seq052_Sequence_Automated_20250711_102529.mat
seq052_Sequence_Automated_20250717_113405.mat
seq052_Sequence_Automated_20250718_121051.mat
seq052_Sequence_Automated_20250722_120114.mat
seq052_Sequence_Automated_20250723_122422.mat


  0%|          | 0/8 [00:00<?, ?it/s]

0
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq052_implant1\recording1_17-06-2025\\\behav_sync\2_task\Preprocessed\\00_20250617_111219_Tue


 12%|█▎        | 1/8 [00:03<00:23,  3.39s/it]

1
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq052_implant1\recording2_18-06-2025\\\behav_sync\2_task\Preprocessed\\01_20250618_113251_Wed


...

 75%|███████▌  | 6/8 [00:17<00:05,  2.99s/it]

6
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq120_implant1\recording8_07-08-2025\\\behav_sync\2_task\Preprocessed\\06_20250807_100828_Thu


 88%|████████▊ | 7/8 [00:20<00:02,  2.95s/it]

7
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq120_implant1\recording9_08-08-2025\\\behav_sync\2_task\Preprocessed\\07_20250808_104807_Fri


100%|██████████| 8/8 [00:23<00:00,  2.92s/it]


In [16]:
Behav_Path

['Sequence_Automated_CurrentTrainingLevel.mat',
 'seq039_Sequence_Automated_20250601_105702.fig',
 'seq039_Sequence_Automated_20250601_105702.mat',
 'seq039_Sequence_Automated_20250602_134508.fig',
 'seq039_Sequence_Automated_20250602_134508.mat',
 'seq039_Sequence_Automated_20250603_084953.fig',
 'seq039_Sequence_Automated_20250603_084953.mat',
 'seq039_Sequence_Automated_20250604_084637.fig',
 'seq039_Sequence_Automated_20250604_084637.mat',
 'seq039_Sequence_Automated_20250604_084646.fig',
 'seq039_Sequence_Automated_20250604_084646.mat',
 'seq039_Sequence_Automated_20250605_081945.fig',
 'seq039_Sequence_Automated_20250605_081945.mat',
 'seq039_Sequence_Automated_20250606_103145.fig',
 'seq039_Sequence_Automated_20250606_103145.mat',
 'seq039_Sequence_Automated_20250607_152647.fig',
 'seq039_Sequence_Automated_20250607_152647.mat',
 'seq039_Sequence_Automated_20250609_111335.fig',
 'seq039_Sequence_Automated_20250609_111335.mat',
 'seq039_Sequence_Automated_20250610_125935.fig',
 '

In [27]:
InputPath = main_paths[Animal_index]              
ephys_dates = Animal_ephys_dates[Animal_index]
'''
Load in all '.mat' files for a given folder if they are ephys sessions and convert them to python format. If there are dulplicate dates, ie. more than one matlab file for a given reocridng 
day, then only keep the largest one:
'''
Behav_Path = sorted(os.listdir(InputPath))
sizes  = []
added_dates = []
to_process = []
for file in Behav_Path:
    if file[-2] == 'a': #if its a .mat and not a fig
        if os.stat(InputPath+ file).st_size > 200000: #more thann 200kb 
            sizes += [os.stat(InputPath+ file).st_size]
            if file != '.DS_Store': #if file is not the weird hidden file 
                
                date = file.split('_')[-2][6:] + '-' + file.split('_')[-2][4:6] + '-' + file.split('_')[-2][:4]
                print('date is: ' + date)
                if date in ephys_dates:
                    if date in added_dates: # code incase there are multiple npoid files for a given recording day - just keep the largest one
                        print('duplicate date')
                        duplicate_index = np.where(np.array(added_dates) ==date)[0][0]
                        # if old duplicate file is larger than current then keep it
                        if sizes[duplicate_index] > sizes[-1]:
                            print('not replacing')
                        else:
                            print('replacing with...')
                            #remove old duplicate 
                            to_process.pop(duplicate_index)
                            added_dates.pop(duplicate_index)
                            sizes.pop(duplicate_index)
                            # replace with new one:
                            print(file)
                            added_dates += [date]
                            to_process += [file]
                    else:
                        print(file)
                        added_dates += [date]
                        to_process += [file]
                        
Behav_Data = {} #set up file dict
File_dates = []             
Sessions = 0 
for file in to_process: 
    Current_file = loadmat(InputPath + file)
    Behav_Data[Sessions] = Current_file
    Sessions = Sessions + 1
    File_dates = File_dates + [file[-19:-4]]


date is: 24-04-2025
date is: 25-04-2025
date is: 28-04-2025
date is: 28-04-2025
date is: 28-04-2025
date is: 05-05-2025
date is: 13-05-2025
date is: 14-05-2025
date is: 17-05-2025
date is: 18-05-2025
date is: 19-05-2025
date is: 20-05-2025
date is: 21-05-2025
date is: 22-05-2025
date is: 23-05-2025
date is: 24-05-2025
date is: 25-05-2025
date is: 26-05-2025
date is: 27-05-2025
date is: 28-05-2025
date is: 29-05-2025
date is: 30-05-2025
date is: 31-05-2025
date is: 01-06-2025


In [32]:
main_paths

['Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq035\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq037\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq038\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq039\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq051\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq052\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq054\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_raw_data\\\\seq059\\Sequence_Automated\\Session D

In [31]:
Behav_Path

['Sequence_Automated_CurrentTrainingLevel.mat',
 'seq051_Sequence_Automated_20250424_113802.fig',
 'seq051_Sequence_Automated_20250424_113802.mat',
 'seq051_Sequence_Automated_20250425_132228.fig',
 'seq051_Sequence_Automated_20250425_132228.mat',
 'seq051_Sequence_Automated_20250428_140229.fig',
 'seq051_Sequence_Automated_20250428_140229.mat',
 'seq051_Sequence_Automated_20250428_141002.fig',
 'seq051_Sequence_Automated_20250428_141002.mat',
 'seq051_Sequence_Automated_20250428_141410.fig',
 'seq051_Sequence_Automated_20250428_141410.mat',
 'seq051_Sequence_Automated_20250429_141504.fig',
 'seq051_Sequence_Automated_20250429_141504.mat',
 'seq051_Sequence_Automated_20250430_132521.fig',
 'seq051_Sequence_Automated_20250430_132521.mat',
 'seq051_Sequence_Automated_20250501_142648.fig',
 'seq051_Sequence_Automated_20250501_142648.mat',
 'seq051_Sequence_Automated_20250502_155242.fig',
 'seq051_Sequence_Automated_20250502_155242.mat',
 'seq051_Sequence_Automated_20250503_100723.fig',
 '

In [30]:
Animal_ephys_dates


[['17-06-2025',
  '18-06-2025',
  '10-07-2025',
  '11-07-2025',
  '17-07-2025',
  '18-07-2025',
  '22-07-2025',
  '23-07-2025'],
 ['11-07-2025',
  '12-07-2025',
  '16-07-2025',
  '17-07-2025',
  '22-07-2025',
  '23-07-2025'],
 ['17-06-2025',
  '18-06-2025',
  '24-06-2025',
  '25-06-2025',
  '09-07-2025',
  '10-07-2025'],
 ['12-08-2025'],
 ['30-07-2025',
  '31-07-2025',
  '01-08-2025',
  '04-08-2025',
  '05-08-2025',
  '06-08-2025',
  '07-08-2025',
  '08-08-2025']]

In [28]:
ephys_dates

['30-07-2025',
 '31-07-2025',
 '01-08-2025',
 '04-08-2025',
 '05-08-2025',
 '06-08-2025',
 '07-08-2025',
 '08-08-2025']

In [42]:
main_paths[Animal_index]

'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\SP156\\Sequence_Automated\\Session Data//'

In [43]:
main_paths

['Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\AP5_1_L\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\AP5_2_R\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\AP5_3_LR\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\SEQ006\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\SEQ007\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\SEQ008\\Sequence_Automated\\Session Data//',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\bpod_data\\\\SP156\\Sequence_Automated\\Session Data//']

In [28]:
Sessions

0

In [27]:
Animal_ephys_dates[Animal_index]

['16-10-2024', '17-10-2024', '22-10-2024', '23-10-2024']